# HNU CE 프로그래밍언어론 (2023년 1학기) HW5
기말과제 파트2

- 이름: 윤승준
- 학번: 20180679

이름과 학번을 위에 작성하시오.


적절한 내용과 개수의 테스트를 작성하지 않으면 감점. 테스트가 적절하지 않거나 지나치게 부족하면 함수를 올바르게 작성했더라도 점수를 얻지 못할 수 있음에 유의!

----

In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables FlexibleInstances TypeApplications RankNTypes KindSignatures #-}

In [2]:
type Nm = String  -- 변수 이름은 문자열로

data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)

-- Let은 제외했음
data Expr = Var Nm                      -- x
          | Lam Nm (Maybe Ty) Expr      -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr      -- rec f:t (\x:t2.e)
          | App Expr Expr               -- (e1 e2)
          | Lit Int                     -- n
          | Add Expr Expr               -- e1 + e2
          | If Expr Expr Expr           -- if e then e1 else e0
          deriving (Eq, Ord)
          
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX
instance IHaskellDisplay Ty   where display = dTeX

In [4]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "
instance {-# OVERLAPS #-} IHaskellDisplay Env    where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay TEnv   where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where display=dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty)  where display=dTeX
instance IHaskellDisplay Val                     where display = dTeX

## 문제 6. (40점) 13장 연습문제 1번,2번,3번,5번
앞선 문제에서 작성했던 것처럼 `infer` 함수의 미완성 부분(`undefined`)을 일단 완성해 놓고 `recon`을 개선하라.

`recon`을 여러 번 작성할 필요는 없고 아래의 `recon` 함수를 그냥 수정하면 된다. 각 연습문제에 해당하는 기능이 구현된 부분에 대해 부분점수를 부여한다.

In [5]:
check tenv e t = infer tenv e == [t]  -- check는 infer로 간단히 정의 가능

infer :: TEnv -> Expr -> [Ty]  -- 적절한 타입이 존재하지 않으면 빈 리스트
infer tenv (Var x)      = case lookup x tenv of Just t  -> [t]
                                                Nothing -> []
infer tenv (Lam x (Just t2) e) = case infer ((x, t2) : tenv) e of
  [t] -> [Arr t2 t]
infer tenv (Lam x Nothing   e) = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (Rec f (Just t) e) = case infer ((f, t) : tenv) e of
  [t'] | t == t' -> [t]
infer tenv (Rec f Nothing  e)  = []  -- 생략된 타입 표기가 있으면 실패
infer tenv (App e1 e2) = case infer tenv e1 of
  [Arr t2 t] -> if check tenv e2 t2 then [t] else []
  _          -> []
infer tenv (Lit _) = [Z]
infer tenv (Add e1 e2) = if check tenv e1 Z && check tenv e2 Z then [Z] else []
infer tenv (If e e1 e0) = case infer tenv e of
  [Z] -> if infer tenv e1 == infer tenv e0 then infer tenv e1 else []
  _   -> []

In [6]:
recon :: TEnv -> Expr -> [Expr]
recon tenv (App (Lam x Nothing e) e2) =
    [ App (Lam x (Just t2) e) e2 | t2 <- infer tenv e2 ]
recon tenv (App (App e1 e2) e2') =
    [ App (App e1' e2) e2' | e1' <- recon tenv e1 ] ++
    [ App e1 e2' | e1 <- recon tenv (App e1 e2), e2' <- recon tenv e2' ]
recon tenv (Lam x mt e) =
    [ Lam x (Just t) e' | t <- infer tenv (Lam x mt e), e' <- recon ((x, t) : tenv) e ]
recon tenv e = [e]
recon _ _ = []

In [7]:
e1 = Lam "x" Nothing . Lam "x'" Nothing $ Var "x" `Add` Var "x'"  
e2 = Lit 2 `Add` Var "y"  -- y는 자유변수!!!
App e1 e2

In [8]:
e2' = Lit 3
App (App e1 e2) e2'

In [9]:
recon [("y",Z)] (App (App e1 e2) e2')